# LSTM

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
df=pd.read_csv("/content/drive/MyDrive/FinallyLemmas.csv")


In [4]:
num_classes=df.label.unique()
num_classes=num_classes.size

In [5]:
X = df.drop(columns='label')
y = df['label']

In [7]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.10, random_state=42) # 10% testing


In [36]:
# encoded_tweets = [tokenizer.encode((sent), add_special_tokens=True) for sent in all_tweets]


# # Find the maximum length
# max_len = max([len(sent) for sent in encoded_tweets])
# print('Max length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (1272 > 512). Running this sequence through the model will result in indexing errors


Max length:  6148


In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size=0.15, random_state=42) # validation 13.5%, Training around 76%


In [10]:
vocab_size = len(tokenizer.word_index) + 1 


In [49]:
vocab_size

216914

In [1]:
max_len = 700
print('Max length: ', max_len)

Max length:  700


In [2]:
from sklearn.utils import shuffle


In [7]:
dff=shuffle(df)

In [8]:
num_classes=dff.label.unique()
num_classes=num_classes.size

In [9]:
X = dff.drop(columns='label')
y = dff['label']

In [10]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.10, random_state=42) # 10% Testing, 13.5% validation, 76% Training


In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train.lemmaText)
X = tokenizer.texts_to_sequences(X_train.lemmaText)
X = pad_sequences(X, maxlen=700)  

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size=0.15, random_state=42)


In [15]:
vocab_size = len(tokenizer.word_index) + 1  

216598

In [16]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=700))
model.add(LSTM(units=256))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10  
batch_size = 32  

history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val))


Epoch 1/10
1388/1388 [==============================] - 271s 192ms/step - loss: 0.9711 - accuracy: 0.6812 - val_loss: 0.4718 - val_accuracy: 0.8691
Epoch 2/10
1388/1388 [==============================] - 126s 91ms/step - loss: 0.2677 - accuracy: 0.9221 - val_loss: 0.2777 - val_accuracy: 0.9268
Epoch 3/10
1388/1388 [==============================] - 98s 70ms/step - loss: 0.1445 - accuracy: 0.9593 - val_loss: 0.1994 - val_accuracy: 0.9442
Epoch 4/10
1388/1388 [==============================] - 78s 56ms/step - loss: 0.0728 - accuracy: 0.9801 - val_loss: 0.1933 - val_accuracy: 0.9480
Epoch 5/10
1388/1388 [==============================] - 79s 57ms/step - loss: 0.0519 - accuracy: 0.9867 - val_loss: 0.1894 - val_accuracy: 0.9551
Epoch 6/10
1388/1388 [==============================] - 74s 53ms/step - loss: 0.0202 - accuracy: 0.9945 - val_loss: 0.2103 - val_accuracy: 0.9526
Epoch 7/10
1388/1388 [==============================] - 72s 52ms/step - loss: 0.0134 - accuracy: 0.9966 - val_loss: 0.213

In [18]:
X_test_sequences = tokenizer.texts_to_sequences(X_test.lemmaText)  
X_test_padded = pad_sequences(X_test_sequences, maxlen=700)  

predictions = model.predict(X_test_padded)

predicted_classes = predictions.argmax(axis=-1)


class_probabilities = predictions


accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Accuracy: {accuracy[1] * 100:.2f}%")


182/182 [==============================] - 3s 15ms/step - loss: 0.2069 - accuracy: 0.9552
Test Accuracy: 95.52%


In [19]:
model.save('LSTM_LastModel.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
